# Prim Benchmark Injection Testing

In [3]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
%run "../../Setup_Scripts/Setup_Generic.ipynb"
%matplotlib widget
scope.adc.samples = 1000

INFO: Found ChipWhisperer😍


In [9]:
scope.dis()
target.dis()

In [4]:
#import necessary library 
import linear_comp as lc
import matplotlib.pyplot as plt 
import chipwhisperer as cw
import numpy as np

In [5]:
def cap_pass_trace():
    scope.arm()
    ret = scope.capture()
    num_char = target.in_waiting()
    while num_char > 0:
        target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    trace = scope.get_last_trace()
    return trace


<h3> This is for Research Benchmark Checking </h3>

In [62]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../hardware/victims/firmware/prim
gmake PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
gmake[1]: '.dep' is up to date.
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
.
Welcome to another exciting ChipWhisperer target build!!
avr-gcc (Homebrew AVR GCC 9.4.0) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     .././hal/xmega/XMEGA_AES_driver.c ...
-e Done!
.
Compiling:
-en     .././hal/xmega/xmega_hal.c ...
-e Done!
.
Compiling:
-en     .././hal/xmega/uart.c ...
-e Done!
.
Compiling:
-en     .././hal/xmega/usart_driver.c ...
-e Done!
.
Compiling:
-en     prim.c ...
-e Done!


prim.c: In function 'primMST':
prim.c:53:9: warning: variable 'parent' set but not used [-Wunused-but-set-variable]
   53 |     int parent[V];
      |         ^~~~~~


.
LINKING:
-en     prim-CWLITEXMEGA.elf ...
-e Done!
.
Creating load file for Flash: prim-CWLITEXMEGA.hex
avr-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature prim-CWLITEXMEGA.elf prim-CWLITEXMEGA.hex
.
Creating load file for Flash: prim-CWLITEXMEGA.bin
avr-objcopy -O binary -R .eeprom -R .fuse -R .lock -R .signature prim-CWLITEXMEGA.elf prim-CWLITEXMEGA.bin
.
Creating load file for EEPROM: prim-CWLITEXMEGA.eep
avr-objcopy -j .eeprom --set-section-flags=.eeprom="alloc,load" \
--change-section-lma .eeprom=0 --no-change-warnings -O ihex prim-CWLITEXMEGA.elf prim-CWLITEXMEGA.eep || exit 0
.
Creating Symbol Table: prim-CWLITEXMEGA.sym
avr-nm -n prim-CWLITEXMEGA.elf > prim-CWLITEXMEGA.sym
.
Creating Extended Listing: prim-CWLITEXMEGA.lss
avr-objdump -h -S -z prim-CWLITEXMEGA.elf > prim-CWLITEXMEGA.lss
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
Size after:
   text	   data	    bss	    dec	    hex	filename
   1990	     54	      2	   2046	    7fe	prim-CWLITEXMEGA.elf
+---------

In [63]:
cw.program_target(scope, prog, "../../hardware/victims/firmware/prim/prim-{}.hex".format(PLATFORM))

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2043 bytes


In [53]:
#this is the method bloc get all the possible traits from the testing, repeatedly. 
reset_target(scope)
trace_set=[]
for i in range(150):
    trace_hit = cap_pass_trace()
    if(len(trace_set))==0:
        trace_set.append(trace_hit)
    else:
        flag=False
        for j in range(len(trace_set)):
            if(lc.linear_comp(trace_hit,trace_set[j],0.1,100))>950:
                flag=True
        
        if not flag:
            trace_set.append(trace_hit)
    
print(len(trace_set))

8


In [ ]:
cw.plot(trace_set[0]) * cw.plot(trace_set[1]) 

# The Testing Part

In [64]:
result=[]
for i in range(7): # we now are getting right result
    reset_target(scope)
    for j in range(120):
        trace_test=cap_pass_trace()
        x=max([lc.linear_comp(trace_test,trace_set[k],0.1,100) for k in range(len(trace_set))])
        result.append(x)

In [65]:
with open("PRIM.csv",'w') as f:
    f.write("Test Iteration,Matching,Decision\n")
    for i in range(len(result)):
        if(result[i]>800):
            f.write(f"{i},{result[i]},OK\n")
        else:
            f.write(f"{i},{result[i]},Failed\n")